In [1]:
import os
import requests
import pandas as pd
import ujson
import dspy
from transformers import pipeline

# Download the RAG corpus if not already present
rag_corpus_url = "https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl"
rag_corpus_path = "ragqa_arena_tech_corpus.jsonl"

if not os.path.exists(rag_corpus_path):
    print(f"Downloading RAG corpus from {rag_corpus_url}...")
    response = requests.get(rag_corpus_url)
    with open(rag_corpus_path, 'wb') as f:
        f.write(response.content)
    print(f"RAG corpus downloaded and saved at {rag_corpus_path}.")

# Truncate long documents in the corpus
max_characters = 6000
with open(rag_corpus_path) as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
print(f"Loaded {len(corpus)} documents from the RAG corpus.")

# Hugging Face Language Model optimized for lightweight deployment
class HuggingFaceLanguageModel(dspy.LM):
    """
    Custom Language Model wrapper for Hugging Face transformers.
    Uses a smaller model optimized for systems with limited resources.
    """
    def __init__(self, model_name='distilgpt2'):  # Smaller model for efficiency
        self.generator = pipeline('text-generation', model=model_name)

    def __call__(self, prompt: str, **kwargs):
        response = self.generator(prompt, max_length=150, **kwargs)[0]['generated_text']
        return dspy.Prediction(text=response)

# Embedder for DSPy Retrievals
embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
retriever = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=5)

# Load CSV data for processing
csv_path = './151_ideas_updated.csv'
df = pd.read_csv(csv_path, usecols=[0, 1, 2, 3, 4, 5])

class DataProcessor(dspy.Module):
    """
    DSPy Module for processing CSV data and generating insights.
    """
    def __init__(self, csv_path: str, column_to_search: str):
        super().__init__()
        self.df = pd.read_csv(csv_path, usecols=[0, 1, 2, 3, 4, 5])
        self.column_to_search = column_to_search
        assert column_to_search in self.df.columns, f"Column '{column_to_search}' not found in CSV."
        self.language_model = HuggingFaceLanguageModel()
        dspy.settings.configure(lm=self.language_model)

    def search_data(self, query: str):
        results = self.df[self.df[self.column_to_search].str.contains(query, case=False, na=False)]
        summary_prompt = f"Summarize {len(results)} results about '{query}': {results.to_string()}"
        summary = self.language_model(summary_prompt).text
        return {
            'query_results': results,
            'result_count': len(results),
            'summary': summary
        }

# RAG Class with Hugging Face Integration
class RAG(dspy.Module):
    """
    RAG model using the CSV and retrievers.
    """
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = retriever(question).passages
        return self.respond(context=context, question=question)

# Initialize DataProcessor and RAG
column_to_search = 'Theme a'  # Adjust as needed
processor = DataProcessor(csv_path=csv_path, column_to_search=column_to_search)
rag = RAG()

# Test DataProcessor
query = "Maximize the Beauty"
simple_result = processor.search_data(query)
print("Search Results:\n", simple_result['query_results'])
print("\nSummary:", simple_result['summary'])

# Test RAG with a query
question = "What are high memory and low memory on Linux?"
rag_response = rag(question)
print("\nRAG Response:", rag_response.response)

# Optimize RAG
tp = dspy.MIPROv2(metric="response_quality", auto="medium", num_threads=4)
trainset = [{"context": "Linux memory management explained.", "question": "What is low memory?", "response": "Low memory refers to memory below 1MB."}]
optimized_rag = tp.compile(RAG(), trainset=trainset, max_bootstrapped_demos=2, max_labeled_demos=2)

# Save and reload optimized RAG
optimized_rag.save("optimized_rag.json")
loaded_rag = RAG()
loaded_rag.load("optimized_rag.json")

# Evaluate the optimized RAG
optimized_response = loaded_rag(question="cmd+tab does not work on hidden or minimized windows")
print("\nOptimized RAG Response:", optimized_response.response)

# Cost Evaluation
cost = sum([x['cost'] for x in dspy.inspect_history() if x['cost'] is not None])
print(f"\nCost (in USD): {cost}")


RAG corpus downloaded and saved at ragqa_arena_tech_corpus.jsonl.
Loaded 28436 documents from the RAG corpus.


AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable